# Step 7: Serve data from OpenAgua into WEAP using WaMDaM

#### By Adel M. Abdallah, Utah State University, Jan 2019

Execute the following cells by pressing `Shift-Enter`, or by pressing the play button <img style='display:inline;padding-bottom:15px' src='play-button.png'> on the toolbar above.



<a name="Import"></a>
# 1. Import python libraries 

In [1]:
# 1. Import python libraries 
### set the notebook mode to embed the figures within the cell
import numpy
import sqlite3
import numpy as np
import pandas as pd
import getpass
from hs_restclient import HydroShare, HydroShareAuthBasic
import os

import plotly
plotly.__version__
import plotly.offline as offline
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
offline.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)         # initiate notebook for offline plot

import os
import csv
from collections import OrderedDict
import sqlite3
import pandas as pd
import numpy as np
from IPython.display import display, Image, SVG, Math, YouTubeVideo
import urllib
import calendar

print 'The needed Python libraries have been imported'

The needed Python libraries have been imported


<a name="ConnectWEAP"></a>
# 2. Connect to the WEAP API

### You need to have WEAP already installed on your machine

First make sure to have a copy of the Water Evaluation And Planning" system (WEAP) installed on your local machine (Windows). If you don’t have it installed, download and install the WEAP software which allows you to run the Bear River WEAP model and its scenarios for Use Case 5. https://www.weap21.org/. You need to have a WEAP License. See here (https://www.weap21.org/index.asp?action=217). If you're interested to learning about WEAP API, check it out here: http://www.weap21.org/WebHelp/API.htm    


## Install dependency and register WEAP
### 2.1. Install pywin32 extensions which provide access to many of the Windows APIs from Python.
**Choose on option**
* a. Install using an executable basedon your python version. Use version for Python 2.7
https://github.com/mhammond/pywin32/releases 

**OR**   

* b. Install it using Anaconda terminal @ https://anaconda.org/anaconda/pywin32

Type this command in the Anaconda terminal as Administrator  

    conda install -c anaconda pywin32 


**OR**

* c. Install from source code (for advanced users) 
https://github.com/mhammond/pywin32


### 2.2. Register WEAP with Windows 


This use case only works on a local Jupyter Notebook server installed on your machine along with WEAP. So it does not work on the online Notebooks in Step 2.1. You need to install Jupyter Server in Step 2.2 then proceed here.




* **Register WEAP with Windows to allow the WEAP API to be accessed**    
Use Windows "Command Prompt". Right click and then <font color=red>**run as Administrator**</font>, navigate to the WEAP installation directory such as and then hit enter  

```
cd C:\Program Files (x86)\WEAP
```

Then type the following command in the command prompt and hit enter   
```
WEAP /regserver
```


<img src="https://github.com/WamdamProject/WaMDaM-software-ecosystem/blob/master/mkdocs/Edit_MD_Files/QuerySelect/images/RegisterWEAP_CMD.png?raw=true" style="float:center;width:700px;padding:20px"> 
Figure 1: Register WEAP API with windows using the Command Prompt (Run as Administrator)


In [23]:
# this library is needed to connect to the WEAP API
import win32com.client

# this command will open the WEAP software (if closed) and get the last active model
# you could change the active area to another one inside WEAP or by passing it to the command here
#WEAP.ActiveArea = "BearRiverFeb2017_V10.9"


WEAP=win32com.client.Dispatch("WEAP.WEAPApplication")

# WEAP.Visible = 'FALSE'


print WEAP.ActiveArea.Name 
# WEAP.ActiveArea = "Weaping River Basin"  
# print WEAP.ActiveArea.Name 

# WEAP.Areas("Weaping River Basin").Open
# WEAP.ActiveArea = "Weaping River Basin"  
print WEAP.ActiveArea.Name


print 'Connected to WEAP API and the '+ WEAP.ActiveArea.Name + ' Area'
print '-------------'
if not WEAP.Registered:
    print "Because WEAP is not registered, you cannot use the API"

# get the active WEAP Area (model) to serve data into it 

# ActiveArea=WEAP.ActiveArea.Name 


# get the active WEAP scenario to serve data into it 
print '-------------'

ActiveScenario= WEAP.ActiveScenario.Name
print '\n ActiveScenario= '+ActiveScenario
print '-------------'

WEAP_Area_dir=WEAP.AreasDirectory
print WEAP_Area_dir


print "\n \n You're connected to the WEAP API"

Weaping River Basin_WaMDaM
Weaping River Basin_WaMDaM
Connected to WEAP API and the Weaping River Basin_WaMDaM Area
-------------
-------------

 ActiveScenario= Reference
-------------
C:\Users\Adel\Documents\WEAP Areas\

 
 You're connected to the WEAP API



# 3. Download the Bear River WEAP model and connect Jupyter Notebook to WEAP API

Clone or download all this GitHub repo   
https://github.com/WamdamProject/WaMDaM_UseCases  

In your local repo folder, go to the    
    
    C:\Users\Adel\Documents\GitHub\WaMDaM_UseCases/UseCases_files/1Original_Datasets_preperation_files/WEAP/Bear_River_WEAP_Model_2017

Copy this folder **Bear_River_WEAP_Model_2017** and paste it into **WEAP Areas** folder on your local machine. For example, it is at   

    C:\Users\Adel\Documents\WEAP Areas  


<a name="CreateWEAP_Area"></a>
# 3.1 Create a copy of the original WEAP Area to use while keeping the orignial as-as for any later use


<a name="AddScenarios"></a>
### Add a new CacheCountyUrbanWaterUse scenario from the Reference original WEAP Area:  

### You can always use this orignal one and delete any new copies you make afterwards.

In [100]:
# Create a copy of the WEAP AREA to serve the updated Hyrym Reservoir to it 

# this library is needed to connect to the WEAP API
import win32com.client

# this command will open the WEAP software (if closed) and get the last active model
# you could change the active area to another one inside WEAP or by passing it to the command here
#WEAP.ActiveArea = "BearRiverFeb2017_V10.9"


WEAP=win32com.client.Dispatch("WEAP.WEAPApplication")

# WEAP.Visible = 'FALSE'


print WEAP.ActiveArea.Name 
# WEAP.ActiveArea = "Weaping River Basin"  
# print WEAP.ActiveArea.Name 

# WEAP.Areas("Weaping River Basin").Open
# WEAP.ActiveArea = "Weaping River Basin"  
print WEAP.ActiveArea.Name


print 'Connected to WEAP API and the '+ WEAP.ActiveArea.Name + ' Area'
print '-------------'
if not WEAP.Registered:
    print "Because WEAP is not registered, you cannot use the API"

# get the active WEAP Area (model) to serve data into it 

# ActiveArea=WEAP.ActiveArea.Name 


# get the active WEAP scenario to serve data into it 
print '-------------'

ActiveScenario= WEAP.ActiveScenario.Name
print '\n ActiveScenario= '+ActiveScenario
print '-------------'

WEAP_Area_dir=WEAP.AreasDirectory
print WEAP_Area_dir


print "\n \n You're connected to the WEAP API"
# Delete the Area if it exists and then add it. Start from fresh
Area="Weaping River Basin_WaMDaM"

if not WEAP.Areas.Exists(Area):
    WEAP.SaveAreaAs(Area)


WEAP.ActiveArea.Save
WEAP.ActiveArea = "Weaping River Basin_WaMDaM"  
print  'ActiveArea= '+ WEAP.ActiveArea.Name

#  Add new Scenario
#  Add(NewScenarioName, ParentScenarioName or Index):   
#  Create a new scenario as a child of the parent scenario specified.    
#  The new scenario will become the selected scenario in the Data View.  
    
    
    
WEAP=win32com.client.Dispatch("WEAP.WEAPApplication")
# WEAP.Visible = FALSE


WEAP.ActiveArea = "Weaping River Basin_WaMDaM"  

print  'ActiveArea= '+ WEAP.ActiveArea.Name

Scenarios=[]
Scenarios=['Demand Measures_WaMDaM']

# Delete the scenario if it exists and then add it. Start from fresh
for Scenario in Scenarios:
    if WEAP.Scenarios.Exists(Scenario):
        # delete it
        WEAP.Scenarios(Scenario).Delete(True)
        # add it back as a fresh copy
        WEAP.Scenarios.Add(Scenario,'Demand Measures')
    else:
        WEAP.Scenarios.Add(Scenario,'Demand Measures')
    
WEAP.ActiveArea.Save
WEAP.SaveArea

WEAP.ActiveArea = "Weaping River Basin_WaMDaM"  

print  'ActiveArea= '+ WEAP.ActiveArea.Name

# or add the scenarios one by one using this command   
    
# Make a copy from the reference (base) scenario
# WEAP.Scenarios.Add('UpdateCacheDemand','Reference')
print '---------------------- \n'
print 'Scenarios added to the original WEAP area'    




print 'Connection with WEAP API is disconnected'

Weaping River Basin_WaMDaM
Weaping River Basin_WaMDaM
Connected to WEAP API and the Weaping River Basin_WaMDaM Area
-------------
-------------

 ActiveScenario= Demand Measures_WaMDaM
-------------
C:\Users\Adel\Documents\WEAP Areas\

 
 You're connected to the WEAP API
ActiveArea= Weaping River Basin_WaMDaM
ActiveArea= Weaping River Basin_WaMDaM
---------------------- 

Scenarios added to the original WEAP area
Connection with WEAP API is disconnected


# Reliability Results: Weeping Model
Reliability is for the entire study period, so doesn't have a year or time step parameter
Reliability to fully meet demand =number of years of zero shortage divided by total number of years

# 5. Run WEAP
<font color=green>**Please wait, it will take ~1-3 minutes** to finish calcualting the two WEAP Areas with their many scenarios</font>

In [ ]:

print 'Please wait 1-3 min for the calculation to finish'
WEAP.Calculate(2006,10,True)
WEAP.SaveArea

print '\n \n The calculation has been done and saved'
print WEAP.CalculationTime

Print 'Done'

In [87]:
Result= OrderedDict()
# columns=['ScenarioName','BranchName','Reliability']
# Result=OrderedDict() 

# group:ScenarioName
# Key=BranchName
# value:BranchName
    
for ScenarioName in WEAP.Scenarios:
    #print ScenarioName
    WEAP.ActiveScenario=ScenarioName

    #print str(ScenarioName)
    scenarioKey = str(ScenarioName)
    Result[scenarioKey] = OrderedDict()
    
    BranchNames = []
    Reliabilities = []
    
    for Branch in WEAP.Branches:
        if Branch.TypeName=='Demand Site' and Branch.IsNode:
            FullBranchName=Branch.FullName
            BranchName=Branch.Name
            if WEAP.Branch(FullBranchName).Variables.Exists("Reliability"):
                #Result['ScenarioName'] = ScenarioName
                BranchNames.append(BranchName)
                Reliabilities.append(WEAP.Branch(FullBranchName).Variables("Reliability").Value)
    Result[scenarioKey]['BranchName'] = BranchNames
    Result[scenarioKey]['Reliability'] = Reliabilities

#     print Result
            
            
 

# Plot reliability

In [97]:


scenario1 = go.Scatter(
    x=Result['Current Accounts']['BranchName'],
    y=Result['Current Accounts']['Reliability'],
    name = 'Current Accounts',
        mode = 'lines+markers',

    marker = dict(
        color = '#264DFF',
    
))


scenario2 = go.Scatter(
    x=Result['Reference']['BranchName'],
    y=Result['Reference']['Reliability'],
    name = 'Reference', 
    mode = 'lines+markers',

    marker = dict(
        color = '#3FA0FF'
))

scenario3 = go.Scatter(
    x=Result['Integrated Measures']['BranchName'],
    y=Result['Integrated Measures']['Reliability'],
    name = 'Integrated Measures',   
    mode = 'lines+markers',

    marker = dict(
        color = '#290AD8'
))

layout = dict(
    #title = "Use Case 3.3",
    yaxis = dict(
        title = "Demand reliability (%)",
        tickformat= ',',
        showline=True,
        tick0=0,

        dtick='10',
        ticks='outside',
        range = [80, 105],
        ticklen=10,
        tickcolor='#000',
        gridwidth=1,
        showgrid=True,

                ),
    
    xaxis = dict(
#         title = "Updated input parameters in the <br>Bear_River_WEAP_Model_2017",
#         showline=True,
        ticks='inside',
         tickfont=dict(size=22),
                tickcolor='#000',
        gridwidth=1,
        showgrid=True,

        ticklen=10
                    ),
    legend=dict(
        x=0.05,y=1.1,
          bordercolor='#00000f',
            borderwidth=2
        
               ),
    width=1100,
    height=700,
    #paper_bgcolor='rgb(233,233,233)',
    #plot_bgcolor='rgb(233,233,233)',
    margin=go.Margin(l=130,b=200),
    font=dict(size=25,family='arial',color='#00000f'),
    showlegend=True
)
data = [scenario1, scenario2,scenario3]


# create a figure object
fig = dict(data=data, layout=layout)
#py.iplot(fig, filename = "2.3Identify_SeasonalValues") 


## it can be run from the local machine on Pycharm like this like below
## It would also work here offline but in a seperate window  
offline.iplot(fig,filename = 'jupyter/UnmentDemand@BirdRefuge' )       

print "Figure x is replicated!!"

Figure x is replicated!!


## Get the unmet demand or Cache County sites in both the reference and the conservation scenarios

In [ ]:
Scenarios=['Reference','Cons25PercCacheUrbWaterUse','Incr25PercCacheUrbWaterUse']
DemandSites=['Logan Potable','North Cache Potable','South Cache Potable']

UnmetDemandEstimate_Ref = pd.DataFrame(columns = DemandSites)
UnmetDemandEstimate_Cons25 = pd.DataFrame(columns = DemandSites)
UnmetDemandEstimate_Incr25 = pd.DataFrame(columns = DemandSites)

UnmetDemandEstimate= pd.DataFrame(columns = Scenarios)

for scen in Scenarios:
        if scen=='Reference':
            for site in DemandSites:
                param="\Demand Sites\%s: Unmet Demand[Acre-Foot]"%(site)
#                 print param
                for year in range (1966,2006):
                    value=WEAP.ResultValue(param, year, 1, scen, year, WEAP.NumTimeSteps)            
                    UnmetDemandEstimate_Ref.loc[year, [site]]=value
        elif scen=='Cons25PercCacheUrbWaterUse':
            for site in DemandSites:
                param="\Demand Sites\%s: Unmet Demand[Acre-Foot]"%(site)
#                 print param
                for year in range (1966,2006):
                    value=WEAP.ResultValue(param, year, 1, scen, year, WEAP.NumTimeSteps)            
                    UnmetDemandEstimate_Cons25.loc[year, [site]]=value
                  
        elif scen=='Incr25PercCacheUrbWaterUse':
            for site in DemandSites:
                param="\Demand Sites\%s: Unmet Demand[Acre-Foot]"%(site)
#                 print param
                for year in range (1966,2006):
                    value=WEAP.ResultValue(param, year, 1, scen, year, WEAP.NumTimeSteps)            
                    UnmetDemandEstimate_Incr25.loc[year, [site]]=value                    
                    
                                             
UnmetDemandEstimate_Ref['Cache Total']=UnmetDemandEstimate_Ref[DemandSites].sum(axis=1)

UnmetDemandEstimate_Cons25['Cache Total']=UnmetDemandEstimate_Cons25[DemandSites].sum(axis=1)

UnmetDemandEstimate_Incr25['Cache Total']=UnmetDemandEstimate_Incr25[DemandSites].sum(axis=1)

UnmetDemandEstimate['Reference']=UnmetDemandEstimate_Ref['Cache Total']
UnmetDemandEstimate['Cons25PercCacheUrbWaterUse']=UnmetDemandEstimate_Cons25['Cache Total']
UnmetDemandEstimate['Incr25PercCacheUrbWaterUse']=UnmetDemandEstimate_Incr25['Cache Total']

UnmetDemandEstimate=UnmetDemandEstimate.rename_axis('Year',axis="columns")

print 'Done estimating the unment demnd pecentage for each scenario'
# display(UnmetDemandEstimate)

# 6. Plot the unmet demad for all the scenarios  and years


In [ ]:


trace2 = go.Scatter(
    x=years,
    y=Reference_vals_perc[0],
    name = 'Reference demand',
        mode = 'lines+markers',

    marker = dict(
        color = '#264DFF',
    
))


trace3 = go.Scatter(
    x=years,
    y=Cons25PercCacheUrbWaterUse_vals_perc[0],
    name = 'Conserve demand by 25%',   
    mode = 'lines+markers',

    marker = dict(
        color = '#3FA0FF'
))

trace1 = go.Scatter(
    x=years,
    y=Incr25PercCacheUrbWaterUse_vals_perc[0],
    name = 'Increase demand by 25%',   
    mode = 'lines+markers',

    marker = dict(
        color = '#290AD8'
))

layout = dict(
    #title = "Use Case 3.3",
    yaxis = dict(
        title = "Annual unmet demand (%)",
        tickformat= ',',
        showline=True,
        dtick='5',
        ticks='outside',
        
        ticklen=10,
        tickcolor='#000',
        gridwidth=1,
        showgrid=True,

                ),
    
    xaxis = dict(
#         title = "Updated input parameters in the <br>Bear_River_WEAP_Model_2017",
#         showline=True,
        ticks='inside',
         tickfont=dict(size=22),
                tickcolor='#000',
        gridwidth=1,
        showgrid=True,

        ticklen=25
                    ),
    legend=dict(
        x=0.05,y=1.1,
          bordercolor='#00000f',
            borderwidth=2
        
               ),
    width=1100,
    height=700,
    #paper_bgcolor='rgb(233,233,233)',
    #plot_bgcolor='rgb(233,233,233)',
    margin=go.Margin(l=130,b=200),
    font=dict(size=25,family='arial',color='#00000f'),
    showlegend=True
)
data = [trace1, trace2,trace3]


# create a figure object
fig = dict(data=data, layout=layout)
#py.iplot(fig, filename = "2.3Identify_SeasonalValues") 


## it can be run from the local machine on Pycharm like this like below
## It would also work here offline but in a seperate window  
offline.iplot(fig,filename = 'jupyter/UnmentDemand@BirdRefuge' )       

print "Figure x is replicated!!"

<a name="Close"></a>
# 7. Close WEAP API connection

In [ ]:
# 9. Close the WEAP API connection


print 'connection disconnected'

# Uncomment 
WEAP.SaveArea



# this command will close WEAP
WEAP.Quit

print 'Connection with WEAP API is disconnected'

# The End :) Congratulations!